In [1]:
import tensorflow as tf
import torch
import torchsummary



2023-07-25 17:01:45.842880: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-25 17:01:45.844371: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-25 17:01:45.874790: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-25 17:01:45.875436: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-25 17:01:46.492892: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
from networks.att_squeeze_unet import AttSqueezeUNet as AttSqueezeUNetTF
model_tf = AttSqueezeUNetTF(2,True)
random_input_tf = tf.random.normal((1, 384, 512, 3))
model_tf(random_input_tf)
model_tf.summary()

Model: "AttSqueezeUNet"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             multiple                  1792      
                                                                 
 max_pooling2d (MaxPooling2  multiple                  0         
 D)                                                              
                                                                 
  (FireModule)               multiple                  11472     
                                                                 
  (FireModule)               multiple                  12496     
                                                                 
 max_pooling2d_1 (MaxPoolin  multiple                  0         
 g2D)                                                            
                                                                 
  (FireModule)               multiple               

In [3]:
import math
def calculate_padding(
        kernel_size, stride, input_size, output_size=None, dilation=(1,1)
    ):
        """Calculate padding for 'same' padding (to have the same output size
        as input).
        """

        if output_size is None:
            output_size = input_size  # assume same padding
        p1 = math.ceil((
            (output_size[0] - 1) * stride[0]
            + 1
            + dilation[0] * (kernel_size[0] - 1)
            - input_size[0]
        ) / 2)
        p2 = math.ceil((
            (output_size[1] - 1) * stride[1]
            + 1
            + dilation[1] * (kernel_size[1] - 1)
            - input_size[1]
        ) / 2)
        return (p1, p2)

In [4]:
def get_same_padding(input_size, kernel_size, stride):
    if input_size % stride == 0:
        pad_along = max(kernel_size - stride, 0)
    else:
        pad_along = max(kernel_size - (input_size % stride), 0)

    pad_left = pad_along // 2
    pad_right = pad_along - pad_left

    return pad_left, pad_right


In [5]:
def calculate_same_padding(input_size, kernel_size, stride, dilation):
        if input_size[0] % stride[0] == 0:
            pad_along_height = max(
                dilation[0] * (kernel_size[0] - stride[0]), 0
            )
        else:
            pad_along_height = max(
                dilation[0] * (kernel_size[0] - (input_size[0] % stride[0])), 0
            )

        if input_size[1] % stride[1] == 0:
            pad_along_width = max(
                dilation[1] * (kernel_size[1] - stride[1]), 0
            )
        else:
            pad_along_width = max(
                dilation[1] * (kernel_size[1] - (input_size[1] % stride[1])), 0
            )

        p1 = math.ceil(pad_along_height/2)
        p2 = math.ceil(pad_along_width/2)
        return (p1, p2)

In [6]:
import torch
from torch import nn
input_shape = (1, 3, 384, 512)

# Desired output shape: (batch_size, out_channels, output_height, output_width)
output_shape = (1, 64, 384, 512)

# Compute the padding required for the same output shape with stride 2x2
kernel_size = (3,3)
stride = (2,2)
dilation = (1,1)
padding = get_same_padding(input_shape[-2], kernel_size[-2], stride[-2])
#padding = calculate_padding(kernel_size, stride, input_shape[-2:], output_shape[-2:],dilation)
padding_1 = calculate_same_padding(input_shape[-2:], kernel_size, stride,dilation=dilation)
print(padding_1==padding)
print("Padding:", padding)
# Define the Conv2d layer
conv_same_output_shape = nn.Conv2d(
    in_channels=input_shape[1],
    out_channels=output_shape[1],
    kernel_size=kernel_size,
    padding=padding_1,
    stride=stride,
    dilation=dilation,
)

# Now, let's see the resulting output shape
input_tensor = torch.randn(input_shape)
output_tensor = conv_same_output_shape(input_tensor)
print("Output shape:", output_tensor.shape)  # 

False
Padding: (0, 1)
Output shape: torch.Size([1, 64, 192, 256])


In [7]:
import torch
import torch.nn as nn

def get_same_padding(input_size, kernel_size, stride):
    if input_size[1] % stride == 0:
        pad_along_height = max(kernel_size - stride, 0)
        pad_along_width = max(
                kernel_size - stride, 0
            )
    else:
        pad_along_height = max(
               kernel_size - (input_size[1] % stride), 0
            )
        pad_along_width = max(kernel_size - (input_size[0] % stride), 0)

    p1 = math.ceil(pad_along_height/2)
    p2 = math.ceil(pad_along_width/2)

    return p1, p2

# Input shape: (batch_size, in_channels, input_height, input_width)
input_shape = (1, 256, 24, 32)

# Desired output shape: (batch_size, out_channels, output_height, output_width)
output_shape = (1, 128, 48, 64)

# Compute the padding required for the ConvTranspose2d layer with "SAME" padding
kernel_size = 3
stride = 2
padding_height = get_same_padding(input_shape[-2:], kernel_size, stride)
padding_width = get_same_padding(input_shape[-2:], kernel_size, stride)

# Define the ConvTranspose2d layer with "SAME" padding
conv_transpose_layer_same_padding = nn.ConvTranspose2d(
    in_channels=input_shape[1],
    out_channels=output_shape[1],
    kernel_size=kernel_size,
    stride=stride,
    padding=padding_height,
    output_padding=1,
)

# Now, let's see the resulting output shape
input_tensor = torch.randn(input_shape)
output_tensor = conv_transpose_layer_same_padding(input_tensor)

print("Output shape:", output_tensor.shape)
print("Padding for height:", padding_height)
print("Padding for width:", padding_width)

Output shape: torch.Size([1, 128, 48, 64])
Padding for height: (1, 1)
Padding for width: (1, 1)


In [11]:
from networks.att_squeeze_unet_torch import AttSqueezeUNet as AttSqueezeUNetTorch
from networks.att_squeeze_unet import AttSqueezeUNet as AttSqueezeUNetTF
height = 256
width = 256
model_torch = AttSqueezeUNetTorch(2, (1, 3,height, width), True)
input_torch = torch.randn((1, 3, height, width))
model_tf = AttSqueezeUNetTF(2,True)
input_tf = tf.random.normal((1, height, width, 3))

In [15]:
input_shape = (1, 64, 32, 32)
padding = ((output_shape[2] - 1) * stride + kernel_size - input_shape[2]) // 2
output_shape = (1, 64, 32, 32)
kernel_size = 3
stride = 2
maxpool_layer = nn.MaxPool2d(kernel_size=kernel_size, stride=stride, padding=padding)
input_tensor = torch.randn(input_shape)
output_tensor = maxpool_layer(input_tensor)

print("Output shape:", output_tensor.shape)  # Output shape: torch.Size([1, 64, 32, 32])

RuntimeError: pad should be at most half of kernel size, but got pad=16 and kernel_size=3

In [4]:
from networks.att_squeeze_unet import UpsamplingBlock as UpsamplingBlockTF
from networks.att_squeeze_unet_torch import UpsamplingBlock as UpsamplingBlockTorch
from torch import nn
tf_upsampling_block = UpsamplingBlockTF(
            filters=64,
            fire_id=11,
            squeeze=16,
            expand=64,
            strides=(2, 2),
            deconv_ksize=3,
            att_filters=16,
        )
random_input_x = tf.random.uniform((1, 24, 32, 256))
random_input_g = tf.random.uniform((1, 48, 64, 128))
print(tf_upsampling_block(random_input_x, random_input_g).shape)
random_input_x_torch = torch.rand((1, 512, 24, 32))
random_input_g_torch = torch.rand((1, 384, 48, 64))
torch_upsampling_block = UpsamplingBlockTorch(
    filters=64,
    fire_id=9,
    squeeze=16,
    expand=64,
    deconv_ksize=3,
    strides=(2, 2),
    att_filters=16,
    x_input_shape=random_input_x_torch.shape,
    g_input_shape=random_input_g_torch.shape,
    )

torch_upsampling_block(random_input_x_torch, random_input_g_torch).shape



(1, 48, 64, 128)
Upconv in shape torch.Size([1, 512, 24, 32])
Upconv shape torch.Size([1, 64, 48, 64])
Attention shape torch.Size([1, 384, 48, 64])
Upconv in shape torch.Size([1, 512, 24, 32])
Upconv shape torch.Size([1, 64, 48, 64])
Attention shape torch.Size([1, 384, 48, 64])


torch.Size([1, 128, 48, 64])

In [10]:
upconv = nn.ConvTranspose2d(
            in_channels=random_input_x_torch.shape[1],
            out_channels=64,
            kernel_size=3,
            stride=(2,2),
            padding=(1, 1),
            # output_padding=(1, 1),
        )
upconv(random_input_x_torch).shape

torch.Size([1, 64, 71, 95])

In [5]:
out = tf_upsampling_block.upconv(random_input_x)
from matplotlib import pyplot as plt
plt.show()

TensorShape([1, 48, 64, 64])

In [ ]:
from networks.att_squeeze_unet import AttentionBlock as AttentionBlockTF 
from networks.att_squeeze_unet_torch import AttentionBlock as AttentionBlockTorch
tf_attention_block = AttentionBlockTF(32)
random_input = tf.random.uniform((1, 256, 256, 32))
random_input_1 = tf.random.uniform((1, 256, 256, 16))
tf_attention_block(random_input_1, random_input)
random_input_torch = torch.rand((1, 32, 256, 256))
random_input_1_torch = torch.rand((1, 16, 256, 256))
torch_attention_block = AttentionBlockTorch(32, 16, 32)
torch_attention_block(random_input_torch, random_input_1_torch)

In [ ]:
from networks.att_squeeze_unet import FireModule as FireModuleTF 
from networks.att_squeeze_unet_torch import FireModule as FireModuleTorch
squeeze = 16
expand = 64
tf_fire_module = FireModuleTF(1, squeeze, expand)
torch_fire_module = FireModuleTorch(1, squeeze, expand)
random_input = tf.random.uniform((1, 256, 256, 3))
random_input_torch = torch.rand((1, 3, 256, 256))
torchsummary.summary(torch_fire_module, (3, 256, 256))
tf_fire_module(random_input)
tf_fire_module.summary()